In [34]:
import sys
sys.path.insert(1, "/Users/pie_trick/Desktop/Semester2/SWengineering/dublin-bikes")    
import pymysql
from sqlalchemy import create_engine
from models.schemas import Base
from config import MySQL
import pandas as pd
import numpy as np
import joblib


host = MySQL.host
user = MySQL.username
password = MySQL.password
database = MySQL.database

engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}/{database}')
Base.metadata.create_all(engine)



number_input = 10
query_bike = "SELECT * \
              FROM development.dublin_bike\
              WHERE number = %(number)s\
              ORDER BY scraping_time desc\
              LIMIT 1"

query_weather = "SELECT * \
                FROM development.current_weather\
                WHERE stationNum =%(number)s \
                ORDER BY datetime desc\
                LIMIT 1"

bike_df = pd.read_sql_query(sql=query_bike, con=engine, params={'number': number_input}, parse_dates=['scraping_time'])
weather_df = pd.read_sql_query(sql=query_weather, con=engine, params={'number': number_input}, parse_dates=['scraping_time'])
# weather_df.to_pickle("./weather_df.pkl")
# bike_df.to_pickle("./bike_df.pkl")

# bike_df = pd.read_pickle("./bike_df.pkl")
# weather_df = pd.read_pickle("./weather_df.pkl")

In [35]:
bike_df

,scraping_time,number,last_update,address,site_names,latitude,longitude,bike_stand,available_bike_stand,available_bike,status,banking,bonus
0,2020-03-29 17:35:01,10,2020-03-29 17:34:16,Dame Street,DAME STREET,53.344,-6.2668,16,3,13,OPEN,1,0


In [36]:
weather_df

,datetime,temperature,description,icon,lon,lat,wind_spd,clouds,sunset,code,weekday,stationNum
0,2020-03-29 17:35:01,4.92,few clouds,02d,-6.27,53.34,5.7,20.0,1585508064,801,7,10


In [37]:
def labeling_eval(df):
    df['available_bike_ratio'] = df['available_bike'] / df['bike_stand']
    return df

In [38]:
def merge_df(bike_df, weather_df):
    combined_df = pd.merge(bike_df, weather_df,  how='inner',\
                           left_on=['number', 'scraping_time'],\
                           right_on = ['stationNum', 'datetime'],\
                           suffixes=('_bike','_weather'))
    return combined_df

In [39]:
def datetime_conversion(df, df_col):
    df['hour'] = df[df_col].dt.hour
    return df

In [40]:
def time_transform(df, col, max_val):
    df[col + '_sin'] = np.sin(2 * np.pi * df[col]/max_val)
    df[col + '_cos'] = np.cos(2 * np.pi * df[col]/max_val)
    df = df.drop([col], axis = 1)
    return df

In [41]:
    def data_cleaning(df):
        pd.set_option('display.max_columns', 500)
        col_to_drop = ['address', 'site_names','bonus', 'last_update', 'datetime','icon', 'lon', 'lat', 'stationNum','available_bike_stand','description','status','sunset']
        df = df.drop(col_to_drop, axis = 1)
        df = df.drop_duplicates().reset_index()
        df = df.drop(['index'] ,axis = 1)
        df = df.drop(['scraping_time','available_bike'] ,axis = 1)

        return df

In [42]:
def data_type_conversion(df):
    categorical_col = ['code']
    df[categorical_col] = df[categorical_col].astype('category')
    df['banking'] = df['banking'].astype('int32')
    df['code'] = df['code'].astype('int32') // 100
    df['temperature'] = df['temperature'].astype('float64')
#     status_map = {'OPEN':1,'CLOSE':0}
#     df['status'] = df['status'].map(status_map)
    return df

In [46]:
bike_df = labeling_eval(bike_df)
combined_df = merge_df(bike_df, weather_df)
combined_df = datetime_conversion(combined_df, "scraping_time")
combined_df = data_type_conversion(combined_df)
combined_df = data_cleaning(combined_df)
combined_df = time_transform(combined_df, 'weekday', 7)
combined_df = time_transform(combined_df, 'hour', 23)
combined_df.columns

Index(['number', 'latitude', 'longitude', 'bike_stand', 'banking',
       'available_bike_ratio', 'temperature', 'wind_spd', 'clouds', 'code',
       'weekday_sin', 'weekday_cos', 'hour_sin', 'hour_cos'],
      dtype='object')

In [47]:
['number', 'latitude', 'longitude', 'bike_stand', 'banking',
       'temperature', 'wind_spd', 'clouds', 'code', 'weekday_sin',
       'weekday_cos', 'hour_sin', 'hour_cos']

['number',
 'latitude',
 'longitude',
 'bike_stand',
 'banking',
 'temperature',
 'wind_spd',
 'clouds',
 'code',
 'weekday_sin',
 'weekday_cos',
 'hour_sin',
 'hour_cos']

In [48]:
len(combined_df.columns)

14

In [49]:
%%time
model_name = 'rf_model.pkl'
model = joblib.load(model_name)
y_prediction = model.predict(combined_df)

CPU times: user 1.79 s, sys: 385 ms, total: 2.17 s
Wall time: 2.3 s


In [50]:
y_prediction = pd.Series(y_prediction)
y_prediction

0    0.196167
dtype: float64